In [55]:
import pandas as pd
pd.set_option('display.width', 1000)
import numpy as np
from datetime import timedelta


#CHECK LINE 20 FOR WHAT TO DO NEXT
#CONVERTING EXIT DATE INTO AGE3DATE IF NECESSARY

In [56]:
exit_notes_df = pd.read_excel('..\Data\TEIS Exit Data for NSS COPY.xlsx', sheet_name='Exit Notes')

In [71]:
teis_data_df = exit_notes_df[exit_notes_df['Exit Note'].str.contains('reopened', case=False, na=False)].reset_index(drop=True)
reopened_df = teis_data_df.drop(columns=['POE', 'Notification Date', 'Initial Eligibility Date', 'Latest IFSP Date'])
#reopened_df.head()

In [72]:
reopened_df['Exit Note'].str.findall('(\\d{2}/\\d{2}/\\d{4})')
dates_df = reopened_df['Exit Note'].str.findall('(\\d{2}/\\d{2}/\\d{4})')
dates_df = dates_df.to_frame()

dates_df['total_dates'] = dates_df['Exit Note'].apply(lambda x: len(str(x).split(' ')))
#display(dates_df)

dates_df.total_dates.describe()

reopened_df['reopened_dates'] = reopened_df['Exit Note'].str.findall('Reopened: \\d{2}/\\d{2}/\\d{4}')
reopened_df['exit_dates'] = reopened_df['Exit Note'].str.findall('Exit Date: \\d{2}/\\d{2}/\\d{4}')

reopen_date_df = reopened_df.reopened_dates.to_frame().astype(str)
exit_date_df = reopened_df.exit_dates.to_frame().astype(str)

reopen_date_df = reopen_date_df.reopened_dates.str.split(',', expand=True)
reopen_date_df.columns = ['reopen1', 'reopen2', 'reopen3', 'reopen4', 'reopen5', 'reopen6']
exit_date_df = exit_date_df.exit_dates.str.split(',', expand=True)
exit_date_df.columns = ['exit1', 'exit2', 'exit3', 'exit4', 'exit5', 'exit6']

cols_r = ['reopen1', 'reopen2', 'reopen3', 'reopen4', 'reopen5', 'reopen6']
cols_e = ['exit1', 'exit2', 'exit3', 'exit4', 'exit5', 'exit6']

reopen_date_df[cols_r] = reopen_date_df[cols_r].replace({'\[':'', '\]':'', '\'':'', 'Reopened:':''}, regex=True)
exit_date_df[cols_e] = exit_date_df[cols_e].replace({'\[':'', '\]':'', '\'':'', 'Exit Date:':''}, regex=True)
#display(reopen_date_df)
#display(exit_date_df)

In [73]:
reopen_exit_date_df = reopen_date_df.join(exit_date_df)

cols_r_e =  ['reopen1', 'reopen2', 'reopen3', 'reopen4', 'reopen5', 'reopen6',
             'exit1', 'exit2', 'exit3', 'exit4', 'exit5', 'exit6']
reopen_exit_date_df = reopen_exit_date_df.astype('datetime64')
reopen_exit_date_df = reopen_exit_date_df.reindex(columns=['exit1', 'reopen1', 'exit2', 'reopen2', 'exit3', 'reopen3', 'exit4', 'reopen4', 'exit5', 'reopen5', 'exit6', 'reopen6'])
reopen_exit_date_df

ifsp_date = reopened_df[['Child ID', 'DOB', 'Initial IFSP Date', 'Exit Date', 'Days from I-IFSP to Exit']]
ifsp_date[['Initial IFSP Date', 'Exit Date']].astype('datetime64')
ifsp_date['age3_date'] = ifsp_date['DOB'] + pd.offsets.DateOffset(years=3)
reopen_exit_date_df = ifsp_date.join(reopen_exit_date_df)

###IF REOPEN DATE IS AFTER 'EXIT DATE', CHANGE 'EXIT DATE' TO CURRENT DAY AND RECALCULATE 'DAYS FROM I-IFSP TO EXIT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen1 > reopen_exit_date_df['Exit Date'], ['Exit Date']] = pd.to_datetime('today').normalize()

###IF 'EXIT DATE' IS AFTER THEIR 3RD BIRTHDAY, CHANGE 'EXIT DATE' TO 'AGE3_DATE'
reopen_exit_date_df['Exit Date'] = np.where(reopen_exit_date_df['Exit Date'] > reopen_exit_date_df.age3_date, reopen_exit_date_df.age3_date, reopen_exit_date_df['Exit Date'])

###RECALCULATE 'DAYS FROM I-IFSP TO EXIT'
reopen_exit_date_df['Days from I-IFSP to Exit'] = reopen_exit_date_df['Exit Date'] - reopen_exit_date_df['Initial IFSP Date']

###SET 'REOPEN1' TO NULL IF THE DATE LISTED IS EQUAL TO OR AFTER 'AGE3_DATE'
###SET 'REOPEN1' TO NULL IF THE DATE LISTED IS EQUAL TO 'EXIT DATE'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen1 > reopen_exit_date_df.age3_date, ['reopen1']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen1 == reopen_exit_date_df.age3_date, ['reopen1']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen1 == reopen_exit_date_df['Exit Date'], ['reopen1']] = 'NaT'




reopen_exit_date_df.loc[reopen_exit_date_df.exit1 < reopen_exit_date_df['Initial IFSP Date'], ['exit1']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.exit2 < reopen_exit_date_df['Initial IFSP Date'], ['exit2']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.exit3 < reopen_exit_date_df['Initial IFSP Date'], ['exit3']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.exit4 < reopen_exit_date_df['Initial IFSP Date'], ['exit4']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.exit5 < reopen_exit_date_df['Initial IFSP Date'], ['exit5']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.exit6 < reopen_exit_date_df['Initial IFSP Date'], ['exit6']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen1 < reopen_exit_date_df['Initial IFSP Date'], ['reopen1']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen2 < reopen_exit_date_df['Initial IFSP Date'], ['reopen2']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen3 < reopen_exit_date_df['Initial IFSP Date'], ['reopen3']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen4 < reopen_exit_date_df['Initial IFSP Date'], ['reopen4']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen5 < reopen_exit_date_df['Initial IFSP Date'], ['reopen5']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen6 < reopen_exit_date_df['Initial IFSP Date'], ['reopen6']] = 'NaT'

reopen_exit_date_df = reopen_exit_date_df[reopen_exit_date_df.exit1.notnull()].reset_index(drop=True)

#reopen_exit_date_df.exit2 = np.where(reopen_exit_date_df.exit2 == reopen_exit_date_df['Exit Date'], reopen_exit_date_df.exit3, reopen_exit_date_df.exit2)
reopen_exit_date_df.exit1 = np.where(reopen_exit_date_df.exit1 == reopen_exit_date_df['Exit Date'], reopen_exit_date_df.exit2, reopen_exit_date_df.exit1)

#reopen_exit_date_df.exit1 = reopen_exit_date_df.exit1.astype('datetime64').dt.date
#reopen_exit_date_df.reopen1 = reopen_exit_date_df.reopen1.astype('datetime64').dt.date

reopen_exit_date_df['r1-e1'] = reopen_exit_date_df.reopen1 - reopen_exit_date_df.exit1
reopen_exit_date_df['r2-e2'] = reopen_exit_date_df.reopen2 - reopen_exit_date_df.exit2
reopen_exit_date_df['r3-e3'] = reopen_exit_date_df.reopen3 - reopen_exit_date_df.exit3
reopen_exit_date_df['r4-e4'] = reopen_exit_date_df.reopen4 - reopen_exit_date_df.exit4
reopen_exit_date_df['r5-e5'] = reopen_exit_date_df.reopen5 - reopen_exit_date_df.exit5
reopen_exit_date_df['r6-e6'] = reopen_exit_date_df.reopen6 - reopen_exit_date_df.exit6
reopen_exit_date_df.dtypes

reopen_exit_date_df['r1-e1'] = reopen_exit_date_df['r1-e1'].fillna(pd.Timedelta(seconds=0))
reopen_exit_date_df['r2-e2'] = reopen_exit_date_df['r2-e2'].fillna(pd.Timedelta(seconds=0))
reopen_exit_date_df['r3-e3'] = reopen_exit_date_df['r3-e3'].fillna(pd.Timedelta(seconds=0))
reopen_exit_date_df['r4-e4'] = reopen_exit_date_df['r4-e4'].fillna(pd.Timedelta(seconds=0))
reopen_exit_date_df['r5-e5'] = reopen_exit_date_df['r5-e5'].fillna(pd.Timedelta(seconds=0))
reopen_exit_date_df['r6-e6'] = reopen_exit_date_df['r6-e6'].fillna(pd.Timedelta(seconds=0))

reopen_exit_date_df['day_diff'] = reopen_exit_date_df['r1-e1'] + reopen_exit_date_df['r2-e2'] + reopen_exit_date_df['r3-e3'] + reopen_exit_date_df['r4-e4'] + reopen_exit_date_df['r5-e5'] + reopen_exit_date_df['r6-e6']
reopen_exit_date_df.day_diff = reopen_exit_date_df.day_diff.astype('timedelta64[D]').astype(int)
reopen_exit_date_df['actual_service_time'] = reopen_exit_date_df['Days from I-IFSP to Exit'].astype('timedelta64[D]').astype(int) - reopen_exit_date_df.day_diff
display(reopen_exit_date_df['actual_service_time'].sort_values())
display(reopen_exit_date_df.iloc[646])
reopen_exit_date_df.dtypes

C:\Users\Alex7\AppData\Local\Temp/ipykernel_12120/4113501961.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ifsp_date['age3_date'] = ifsp_date['DOB'] + pd.offsets.DateOffset(years=3)


548      37
321      37
261      48
16       51
374      61
       ... 
646    1022
181    1025
194    1036
607    1056
563    1056
Name: actual_service_time, Length: 723, dtype: int32

Child ID                                 424087
DOB                         2017-08-10 00:00:00
Initial IFSP Date           2017-10-23 00:00:00
Exit Date                   2020-08-10 00:00:00
Days from I-IFSP to Exit     1022 days 00:00:00
age3_date                   2020-08-10 00:00:00
exit1                       2020-07-20 00:00:00
reopen1                                     NaT
exit2                                       NaT
reopen2                                     NaT
exit3                                       NaT
reopen3                                     NaT
exit4                                       NaT
reopen4                                     NaT
exit5                                       NaT
reopen5                                     NaT
exit6                                       NaT
reopen6                                     NaT
r1-e1                           0 days 00:00:00
r2-e2                           0 days 00:00:00
r3-e3                           0 days 0

Child ID                              int64
DOB                          datetime64[ns]
Initial IFSP Date            datetime64[ns]
Exit Date                    datetime64[ns]
Days from I-IFSP to Exit    timedelta64[ns]
age3_date                    datetime64[ns]
exit1                        datetime64[ns]
reopen1                      datetime64[ns]
exit2                        datetime64[ns]
reopen2                      datetime64[ns]
exit3                        datetime64[ns]
reopen3                      datetime64[ns]
exit4                        datetime64[ns]
reopen4                      datetime64[ns]
exit5                        datetime64[ns]
reopen5                      datetime64[ns]
exit6                        datetime64[ns]
reopen6                      datetime64[ns]
r1-e1                       timedelta64[ns]
r2-e2                       timedelta64[ns]
r3-e3                       timedelta64[ns]
r4-e4                       timedelta64[ns]
r5-e5                       time

In [123]:
service_time_df = reopen_exit_date_df[['Child ID', 'actual_service_time']]

initial_final_merge = exit_notes_df.merge(service_time_df, how='outer')
initial_final_merge.actual_service_time = initial_final_merge.actual_service_time.fillna(initial_final_merge['Days from I-IFSP to Exit']).astype(int)

initial_final_merge['change_bool'] = np.where(initial_final_merge.actual_service_time == initial_final_merge['Days from I-IFSP to Exit'], 0, 1).astype(bool)
initial_final_merge['service_change'] = (initial_final_merge['Days from I-IFSP to Exit'] - initial_final_merge.actual_service_time).abs()

service_time_stats_df = initial_final_merge['service_change']
service_time_stats_df = service_time_stats_df.to_frame()
service_time_stats_df = service_time_stats_df[service_time_stats_df.service_change != 0].reset_index(drop=True)
service_time_stats_df.describe()


#initial_final_merge.head(20)

##Add column to see if the service days changed
##Add column to show how much it changed if it did
##Calculate how many children had changed service days

,service_change
count,669.000000
mean,204.106129
std,173.026890
min,1.000000
25%,64.000000
50%,159.000000
75%,314.000000
max,767.000000
